## Preprocessing
Experiment: Effect of suicide rates on life expectancy

### Environment Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
LIFE_EXPECTANCY_DATA = "../10_Data/10_life_expectancy_v00_orig.csv"
WHO_SUICIDE_DATA = "../10_Data/20_suicide_rates_v00_orig.csv"

### Dataset loading and basic preprocessing

#### 1. Loading Life Expectancy dataset

In [3]:
lifeExpDf = pd.read_csv(LIFE_EXPECTANCY_DATA)
lifeExpDf.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

Choose only variables which are interesting for the experiment, rename variables, change type of year and set index.

In [4]:
lifeExpDf = lifeExpDf[['Country', 'Year', 'Life expectancy ', 'Adult Mortality', 'infant deaths', 'Alcohol', 'under-five deaths ', ' HIV/AIDS', 'GDP', 'Population', 'Income composition of resources', 'Schooling']]
lifeExpDf = lifeExpDf.rename(columns={'Life expectancy ': 'Life expectancy', 'infant deaths': 'Infant deaths', 'under-five deaths ': 'Under-five deaths', ' HIV/AIDS': 'HIV/AIDS'})
lifeExpDf = lifeExpDf.astype({'Year':'str'})
lifeExpDf = lifeExpDf.set_index(['Country', 'Year'])

In [5]:
lifeExpDf.head()

Life expectancy  Adult Mortality  Infant deaths  Alcohol  \
Country     Year                                                             
Afghanistan 2015             65.0            263.0             62     0.01   
            2014             59.9            271.0             64     0.01   
            2013             59.9            268.0             66     0.01   
            2012             59.5            272.0             69     0.01   
            2011             59.2            275.0             71     0.01   

                  Under-five deaths  HIV/AIDS         GDP  Population  \
Country     Year                                                        
Afghanistan 2015                 83       0.1  584.259210  33736494.0   
            2014                 86       0.1  612.696514    327582.0   
            2013                 89       0.1  631.744976  31731688.0   
            2012                 93       0.1  669.959000   3696958.0   
            2011                 97       0.1   63.537231   2978599.0   

                  Income composition of resources  Schooling  
Country     Year                                              
Afghanistan 2015                            0.479       10.1  
            2014                            0.476       10.0  
            2013                            0.470        9.9  
            2012                            0.463        9.8  
            2011                            0.454        9.5

In [6]:
lifeExpDf.dtypes

Life expectancy                    float64
Adult Mortality                    float64
Infant deaths                        int64
Alcohol                            float64
Under-five deaths                    int64
HIV/AIDS                           float64
GDP                                float64
Population                         float64
Income composition of resources    float64
Schooling                          float64
dtype: object

In [7]:
lifeExpDf.shape

(2938, 10)

In [8]:
# Export final dataset
lifeExpDf.to_csv('../10_Data/10_life_expectancy_v01_clean.csv', index = True)

#### 2. Loading Suicide dataset

In [9]:
suicideDf = pd.read_csv(WHO_SUICIDE_DATA)
suicideDf.head()

,country,year,sex,age,suicides_no,population
0,Albania,1985,female,15-24 years,NaN,277900.0
1,Albania,1985,female,25-34 years,NaN,246800.0
2,Albania,1985,female,35-54 years,NaN,267500.0
3,Albania,1985,female,5-14 years,NaN,298300.0
4,Albania,1985,female,55-74 years,NaN,138700.0


Drop the sex and age columns, rename varriables, change type of the year variable and set index.

In [10]:
suicideDf = suicideDf.drop(columns=['sex', 'age'])
suicideDf = suicideDf.rename(columns={'country': 'Country', 'year': 'Year', 'suicides_no': 'Suicides number', 'population': 'Population2'})
suicideDf = suicideDf.astype({'Year':'str'})
suicideDf = suicideDf.set_index(['Country', 'Year'])
suicideDf.head()

Suicides number  Population2
Country Year                              
Albania 1985              NaN     277900.0
        1985              NaN     246800.0
        1985              NaN     267500.0
        1985              NaN     298300.0
        1985              NaN     138700.0

Drop NA if in the Suicides number variable.

In [11]:
suicideDf = suicideDf.dropna(subset = ['Suicides number'])

Group suicides number by country and year.

In [12]:
suicideDf = suicideDf.groupby(by = ["Country", "Year"]).sum()

In [13]:
suicideDf.head()

Suicides number  Population2
Country Year                              
Albania 1987             73.0    2709600.0
        1988             63.0    2764300.0
        1989             68.0    2803100.0
        1992             47.0    2822500.0
        1993             73.0    2807300.0

In [14]:
suicideDf.dtypes

Suicides number    float64
Population2        float64
dtype: object

In [15]:
suicideDf.shape

(3460, 2)

In [16]:
# Export final dataset
suicideDf.to_csv('../10_Data/20_suicide_rates_v01_clean.csv', index = True)

#### Merge datasets

In [17]:
mergedDf = suicideDf.merge(lifeExpDf, left_index = True, right_index = True)
mergedDf.head()

Suicides number  Population2  Life expectancy  Adult Mortality  \
Country Year                                                                   
Albania 2000             54.0    2796300.0             72.6             11.0   
        2001            119.0    2799349.0             73.6             14.0   
        2002            133.0    2818839.0             73.3             15.0   
        2003            124.0    2843929.0             72.8             18.0   
        2004            146.0    2874991.0             73.0             17.0   

              Infant deaths  Alcohol  Under-five deaths  HIV/AIDS  \
Country Year                                                        
Albania 2000              1     3.66                  1       0.1   
        2001              1     4.25                  1       0.1   
        2002              1     3.73                  1       0.1   
        2003              1     4.29                  1       0.1   
        2004              1     4.54                  1       0.1   

                      GDP  Population  Income composition of resources  \
Country Year                                                             
Albania 2000  1175.788981     38927.0                            0.656   
        2001  1326.973390     36173.0                            0.662   
        2002  1453.642777      3511.0                            0.670   
        2003   189.681557    339616.0                            0.674   
        2004  2416.588235    326939.0                            0.681   

              Schooling  
Country Year             
Albania 2000       10.7  
        2001       10.6  
        2002       10.7  
        2003       10.7  
        2004       10.9

In [18]:
mergedDf.shape

(1424, 12)

In [19]:
mergedDf.dtypes

Suicides number                    float64
Population2                        float64
Life expectancy                    float64
Adult Mortality                    float64
Infant deaths                        int64
Alcohol                            float64
Under-five deaths                    int64
HIV/AIDS                           float64
GDP                                float64
Population                         float64
Income composition of resources    float64
Schooling                          float64
dtype: object

In [20]:
mergedDf.index.unique(level='Year')

Index(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2012', '2013', '2014', '2015', '2011'],
      dtype='object', name='Year')

#### Dealing with missing values

In [21]:
mergedDf.isnull().sum()

Suicides number                      0
Population2                          0
Life expectancy                      1
Adult Mortality                      1
Infant deaths                        0
Alcohol                             61
Under-five deaths                    0
HIV/AIDS                             0
GDP                                139
Population                         296
Income composition of resources     48
Schooling                           48
dtype: int64

Take population from population2 if NA and drop population2

In [22]:
mergedDf['Population'] = mergedDf.apply(
    lambda row: row['Population2'] if np.isnan(row['Population']) else row['Population'],
    axis = 1
)
mergedDf = mergedDf.drop(columns=['Population2'])

Drop rows where life expectancy is NA and population is zero

In [23]:
mergedDf = mergedDf.dropna(subset = ['Life expectancy'])
mergedDf = mergedDf[(mergedDf['Population'] != 0)]

Fill remaining missing values with forward values

In [24]:
mergedDf = mergedDf.fillna(method='ffill')
mergedDf.isnull().sum()

Suicides number                    0
Life expectancy                    0
Adult Mortality                    0
Infant deaths                      0
Alcohol                            0
Under-five deaths                  0
HIV/AIDS                           0
GDP                                0
Population                         0
Income composition of resources    0
Schooling                          0
dtype: int64

In [25]:
# Export final dataset
mergedDf.to_csv('../10_Data/30_merged_dataset_v00_final.csv', index = True)